# QPAM |👷🏻‍♀️>: Q# Project Architect Manager

In [1]:
from qbob import qbob, qpam
from qbob.intrinsics import Message, M, H

#### Create HelloWorld operation

In [2]:
my_qbob = qbob.OperationBuilder("HelloWorld", entrypoint=True, debug=True)
my_qbob += Message("Hello World!")
with my_qbob.allocate_qubits("q", 1) as q:
    my_qbob += H(q)
    my_qbob.log_state(q)
    my_qbob.returns(M(q))

#### Generate project

In [3]:
my_qpam = qpam.ProgramArchitect("MyProject", executable=True)
my_qpam.add_operations(my_qbob)
project_file = my_qpam.create_project(".")

Hello World!


In [4]:
project_file

'MyProject.csproj'

### Create project from multiple operation builders

#### Create Teleport operation

In [5]:
from qbob.intrinsics import H, CNOT, X, Z, Measure
from qbob.types import Qubit, Pauli, One

teleport = qbob.OperationBuilder("Teleport")
msg = teleport.input("msg", Qubit)
target = teleport.input("target", Qubit)

with teleport.allocate_qubits("auxiliary", 1) as aux:
    teleport += H(aux)
    teleport += CNOT(aux, target)
    teleport += CNOT(msg, aux)
    teleport += H(msg)

    with teleport.if_statement(Measure([Pauli.PauliZ], [msg]) == One):
        teleport += Z(target)

    with teleport.if_statement(Measure([Pauli.PauliZ], [aux]) == One):
        teleport += X(target)

print(teleport.build())

operation Teleport (msg : Qubit, target : Qubit) : Unit {
    using (auxiliary = Qubit()) {
        H(auxiliary);
        CNOT(auxiliary, target);
        CNOT(msg, auxiliary);
        H(msg);

        if (Measure([PauliZ], [msg]) == One) {
            Z(target);
        }

        if (Measure([PauliZ], [auxiliary]) == One) {
            X(target);
        }
    }
}


#### Create TeleportMessage entrypoint operation

In [6]:
my_qbob = qbob.OperationBuilder("TeleportMessage", entrypoint=True)
message = my_qbob.input("message", bool)

with my_qbob.allocate_qubits("register", 2) as register:
    msg = register[0]
    target = register[1]

    with my_qbob.if_statement(message == True):
        my_qbob += X(msg)
    
    my_qbob += H(msg)
    my_qbob += teleport(msg, target)    
    my_qbob.returns(Measure([Pauli.PauliX], [target]))

print(my_qbob.build())

@EntryPoint()
operation TeleportMessage (message : Bool) : Result {
    using (register = Qubit[2]) {

        if (message == true) {
            X(register[0]);
        }
        H(register[0]);
        Teleport(register[0], register[1]);
        return Measure([PauliX], [register[1]]);
    }
}


#### Generate project

In [7]:
my_qpam = qpam.ProgramArchitect("TeleportMessage", executable=True)
my_qpam.add_operations(teleport, my_qbob)
project_file = my_qpam.create_project("teleport")